In [ ]:
# !pip install -r requirements.txt 

# This is the full library requirements from my virtualenv including jupyter notebook. 
# If this is once executed, no need to run it here

In [1]:
import os
import sys
import json
import datetime
import skimage.io
import numpy as np
import skimage.draw
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
import glob
import random, csv, json
from scipy import ndimage
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Root directory of the project
ROOT_DIR = os.path.join(os.getcwd(),"Mask_RCNN-2.1")
%cd Mask_RCNN-2.1
from config import Config
import utils
import model as modellib

print(ROOT_DIR)

/home/wahlau/Programming/deepspray-seg-tf1.3/compress/Mask_RCNN-2.1


Using TensorFlow backend.


/home/wahlau/Programming/deepspray-seg-tf1.3/compress/Mask_RCNN-2.1


In [3]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

print(COCO_WEIGHTS_PATH)
print(DEFAULT_LOGS_DIR)

/home/wahlau/Programming/deepspray-seg-tf1.3/compress/Mask_RCNN-2.1/mask_rcnn_coco.h5
/home/wahlau/Programming/deepspray-seg-tf1.3/compress/Mask_RCNN-2.1/logs


In [4]:
class SegmentationConfig(Config):
  """Configuration for training on the toy  dataset.
  Derives from the base Config class and overrides some values.
  """
  # Give the configuration a recognizable name
  NAME = "deepspray"

  # We use a GPU with 12GB memory, which can fit two images.
  # Adjust down if you use a smaller GPU.
  IMAGES_PER_GPU = 2

  # Number of classes (including background)
  NUM_CLASSES = 1 + 1  # Background + dech-lig

  # Number of training steps per epoch
  STEPS_PER_EPOCH = 100

  # Skip detections with < 90% confidence
  DETECTION_MIN_CONFIDENCE = 0.9

############################################################
#  Dataset
############################################################

class SegmentationDataset(utils.Dataset):

  def load_dataset(self, dataset_dir, subset):
    """Load a subset of the deepspray dataset.
    dataset_dir: Root directory of the dataset.
    subset: Subset to load: train or val
    """
    # Add classes. We have only one class to add.
    self.add_class("detchlgm", 1, "detchlgm")

    # Train or validation dataset?
    assert subset in ["train", "valid"]

    if(subset == "train"):
      num_image_set = 1000
      label_file = os.path.join(dataset_dir, "labels_train_green_30_10_2021.json")
    else:
      num_image_set = 200
      label_file = os.path.join(dataset_dir, "labels_valid_green_30_10_2021.json")

    dataset_dir = os.path.join(dataset_dir, subset)
    annotations = json.load(open(label_file))

    for i in range (num_image_set):
      imgurl = os.path.join(dataset_dir, str(i)+".png")
      image = skimage.io.imread(imgurl)
      height, width = image.shape[:2]

      dc_from_json = annotations[i]
      polygons = []
      for j, p in enumerate(dc_from_json):
        p[1] = [j*height for j in p[1]]
        p[0] = [j*width for j in p[0]]
        polygons.append([p[0],p[1]])

      self.add_image(
          "detchlgm",
          image_id=str(i)+".png",  # use file name as a unique image id
          path=imgurl,
          width=width, height=height,
          polygons=polygons)

  def load_mask(self, image_id):
    """Generate instance masks for an image.
    Returns:
    masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
    class_ids: a 1D array of class IDs of the instance masks.
    """
    # If not a balloon dataset image, delegate to parent class.
    image_info = self.image_info[image_id]
    if image_info["source"] != "detchlgm":
        return super(self.__class__, self).load_mask(image_id)

    # Convert polygons to a bitmap mask of shape
    # [height, width, instance_count]
    info = self.image_info[image_id]
    mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                    dtype=np.uint8)
    for i, p in enumerate(info["polygons"]):
        # Get indexes of pixels inside the polygon and set them to 1
        rr, cc = skimage.draw.polygon(p[1], p[0])
        mask[rr, cc, i] = 1

    # Return mask, and array of class IDs of each instance. Since we have
    # one class ID only, we return an array of 1s
    return mask, np.ones([mask.shape[-1]], dtype=np.int32)

  def image_reference(self, image_id):
    """Return the path of the image."""
    info = self.image_info[image_id]
    if info["source"] == "detchlgm":
        return info["path"]
    else:
        super(self.__class__, self).image_reference(image_id)

def load_dataset_images(dataset):
  # Train or evaluate
  dataset_train = SegmentationDataset()
  dataset_train.load_dataset(dataset, "train")
  dataset_train.prepare()

  # Validation dataset
  dataset_val = SegmentationDataset()
  dataset_val.load_dataset(dataset, "valid")
  dataset_val.prepare()
  return dataset_train,dataset_val

In [5]:
config = SegmentationConfig()
config.display()
command = "train"
logs = DEFAULT_LOGS_DIR
weights = "coco"

if command == "train":
  model = modellib.MaskRCNN(mode="training", config=config, model_dir=logs)
else:
  model = modellib.MaskRCNN(mode="inference", config=config, model_dir=logs)

if weights.lower() == "coco":
  weights_path = COCO_WEIGHTS_PATH
    # Download weights file
    # if not os.path.exists(weights_path):
    #     utils.download_trained_weights(weights_path)
elif weights.lower() == "last":
    # Find last trained weights
  weights_path = model.find_last()[1]
elif weights.lower() == "imagenet":
    # Start from ImageNet trained weights
  weights_path = model.get_imagenet_weights()
else:
  weights_path = weights

# Load weights
print("Loading weights ", weights_path)
if weights.lower() == "coco":
    # Exclude the last layers because they require a matching
    # number of classes
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])
else:
    model.load_weights(weights_path, by_name=True)



Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           deepspray
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFE

In [6]:
dataset_train,dataset_val = load_dataset_images("dataset")

In [ ]:
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

# this should be done. The trained weights are inside the logs directory. 


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/wahlau/Programming/deepspray-seg-tf1.3/compress/Mask_RCNN-2.1/logs/deepspray20211104T2302/mask_rcnn_deepspray_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)